In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import collections
import os

C:\Users\Anand\anaconda3\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [5]:
IMAGE_PATH = '../data/img_sz_512/'
csv_path = '../data/train.csv'

In [12]:
df = pd.read_csv(csv_path)

def load_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def init_grabcut_mask(h, w):
    mask = np.ones((h, w), np.uint8) * cv2.GC_PR_BGD
    mask[h//4:3*h//4, w//4:3*w//4] = cv2.GC_PR_FGD
    mask[2*h//5:3*h//5, 2*w//5:3*w//5] = cv2.GC_FGD
    return mask

def add_contours(image, mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if len(contours) != 0:
        cv2.drawContours(image, contours, -1, (255, 0, 0), 3)
        c = max(contours, key = cv2.contourArea)
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0) ,2)
        
def remove_background(image):
    h, w = image.shape[:2]
    mask = init_grabcut_mask(h, w)
    bgm = np.zeros((1, 65), np.float64)
    fgm = np.zeros((1, 65), np.float64)
    cv2.grabCut(image, mask, None, bgm, fgm, 1, cv2.GC_INIT_WITH_MASK)
    mask_binary = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    result = cv2.bitwise_and(image, image, mask = mask_binary)
#     add_contours(result, mask_binary) # optional, adds visualizations
    return result

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

In [7]:
df.head()

,image,labels
0,800113bb65efe69e.jpg,healthy
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex
2,80070f7fb5e2ccaa.jpg,scab
3,80077517781fb94f.jpg,scab
4,800cbf0ff87721f8.jpg,complex


In [13]:
# if not os.path.exists('resized_images'):
#     os.makedirs('resized_images')
#     os.makedirs('resized_images/resized_256')
#     os.makedirs('resized_images/resized_512')
#     os.makedirs('resized_images/resized_1024')
    
# if not os.path.exists('preprocessed_images'):
#     os.makedirs('preprocessed_images')
#     os.makedirs('preprocessed_images/preprocessed_images_256')
#     os.makedirs('preprocessed_images/preprocessed_images_512')
#     os.makedirs('preprocessed_images/preprocessed_images_1024')

if not os.path.exists('../data/preprocessed_images_512'):
    os.makedirs('../data/preprocessed_images_512')

In [17]:
train_filenames = [os.path.join(IMAGE_PATH, fname) for fname in df['image'].tolist()]

for i in tqdm(range(len(train_filenames))):
    fname = train_filenames[i]
    img_orig = plt.imread(fname)
    
    #############################
    for h in [512]:
        img_name = fname.split('/')[-1]
        img = image_resize(img_orig, height=h)
        img_preprocessed = remove_background(img)
#         cv2.imwrite(os.path.join('resized_images/resized_{}'.format(h), img_name), img)
#         cv2.imwrite(os.path.join('preprocessed_images/preprocessed_images_{}'.format(h), img_name), img_preprocessed)
        cv2.imwrite(os.path.join('../data/preprocessed_images_512', img_name), cv2.cvtColor(img_preprocessed, cv2.COLOR_BGR2RGB))
    


100%|██████████| 18632/18632 [6:02:15<00:00,  1.17s/it]  
